In [1]:
!pip install spacy && python -m spacy download en
!pip install sentence-transformers

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 2.3MB 35.1MB/s 
     |████████████████████████████████| 1.2MB 47.3MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 31.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=7455441e3ba30ff0bd62a3e545a1b484ac0316b9827641eccfba8533f3cec540
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [2]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

In [3]:
from pathlib import Path
import os, re, io
import pandas as pd
import numpy as np
import requests

from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.metrics.pairwise import cosine_similarity

import random
import pickle

from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
folder="/content/drive/MyDrive/Sem 3-2 Online/Question Answering System/Final/Code/1-5 DistilBERT"

In [6]:
qA_model=pickle.load(open(folder+'/qA_model.sav', 'rb'))
qQ_model=pickle.load(open(folder+'/qQ_model.sav', 'rb'))

In [7]:
class TextPreprocessor():
    def __init__(self, data_df, column_name=None):
        self.data_df = data_df  
        if not column_name and type(colum_name) == str:
            raise Exception("column name is mandatory. Make sure type is string format")
        self.column = column_name
        self.convert_lowercase()    
        self.applied_stopword = False
        self.processed_column_name = f"processed_{self.column}"
        
    def convert_lowercase(self):
        self.data_df.fillna('',inplace=True)
        self.data_df = self.data_df.apply(lambda column: column.astype(str).str.lower(), axis=0)    

    def remove_question_no(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'^\d+[.]',' ', row))    
        
    def remove_symbols(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'[^A-Za-z0-9\s]', ' ', row))    

    def process(self):
        self.remove_question_no()
        self.remove_symbols()  
        return self.data_df

In [8]:
def similarity_score(u, v):
 return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [9]:
def rank_results(qQ_scores, qA_scores,query):
  # qQ_top10=sorted(list(enumerate(qQ_scores)), key=lambda x: x[1])[-100:]
  # index= [i for i,score in qQ_top10]
  top = [(i,0.20*qA_scores[i]+0.80*qQ_scores[i]) for i in range(len(qA_scores))] 
  ranked=sorted(top, key=lambda x: x[1])

  index_ranked=[i for (i,score) in ranked][::-1][:100]

  faqID_ranked=list(QA_df.FaqID.iloc[index_ranked])

  final_list = bm25_similarity(faqID_ranked,query)

  for i in range(1,6):
    print("Retrieved Answer "+str(i)+" : "+ list(QA_df.iloc[index_ranked]['FaqAnswer'])[i-1])
  # print(faqID_ranked)
  return final_list

In [10]:
def bm25_similarity(faqid_ranked,query):
  corpus = list(QA_df[QA_df['FaqID'].isin(faqid_ranked)]['FaqQuestion'])

  tokenized_corpus = [doc.split(" ") for doc in corpus]
  bm25 = BM25Okapi(tokenized_corpus)

  tokenized_query = query.split(" ")
  top_50 = bm25.get_top_n(tokenized_query, corpus, n=50)
  print(type(top_50[0]))

  list_top_50 = [QA_df.loc[QA_df['FaqQuestion'] == i , 'FaqID'].tolist()[0] for i in top_50]
  #list_top_50 = df.query('B==3')['A']

  print(top_50)
  return list_top_50


In [11]:
def find_relevance_score(faqID_ranked, query_grpID):
  relevance_df=pd.read_csv(folder+'/faqir_relevance.csv')
  val_list=[]
  #print((faqID_ranked))
  for faqID in faqID_ranked:
    #print(type(faqID))
    if (relevance_df['Val'][(relevance_df['FaqID'] == faqID) & (relevance_df['GrpID'] == query_grpID)].empty ==False):
      val_list.append(relevance_df['Val'][(relevance_df['FaqID'] == faqID) & (relevance_df['GrpID'] == query_grpID)].values[0])
    else:
      val_list.append(5)
  return val_list

In [12]:
def performance_metrics(relevance_list):
  p5=0.0
  rr=0.0
  ap=0.0

  if(1 in relevance_list):
    p5=relevance_list[:5].count(1)/5.0
    rr=1.0/(relevance_list.index(1)+1)

    ct=0
    b=[]
    for i in range(len(relevance_list)):
      if relevance_list[i] ==1:
        ct=ct+1
      b.append(ct/(float)(i+1))
    index_1=[i for i in range(len(relevance_list)) if relevance_list[i] == 1] 
    ap=sum([b[i] for i in index_1])/ct

  
  return p5,ap,rr

In [13]:
processed_QA_df=pd.read_csv(folder+'/processed_QA_df.csv',index_col=0)
QA_df = pd.read_csv(folder+'/faqir_faq.csv',header=[0],index_col=0)

In [14]:
processed_QA_df['Q_encoding']=[qQ_model.encode(Q) for Q in processed_QA_df['FaqQuestion']]
processed_QA_df['A_encoding']=[qA_model.encode(A) for A in processed_QA_df['FaqAnswer']]

In [15]:
text_preprocessor_query = TextPreprocessor(pd.read_csv(folder+'/faqir_query.csv', index_col=0), column_name=['Query'])
test_data=text_preprocessor_query.process()
test_data = test_data.astype({'Query_GrpID':int}) 

In [16]:
query_df=pd.read_csv(folder+'/faqir_query.csv', index_col=0)

In [ ]:
m_p5=[]
m_ap=[]
m_rr=[]
for i in test_data.index:
  q=test_data['Query'][i]
  print("User Query : "+query_df.iloc[i]['Query'])
  # print(test_data['Query_GrpID'][i])
  qQ_query_encode=qQ_model.encode(q)
  qA_query_encode=qA_model.encode(q)
  qQ_scores=[similarity_score(qQ_query_encode,Q_encode) for Q_encode in processed_QA_df['Q_encoding']]
  qA_scores=[similarity_score(qA_query_encode,A_encode) for A_encode in processed_QA_df['A_encoding']]
  relevance_list=find_relevance_score(rank_results(qQ_scores, qA_scores, q), test_data['Query_GrpID'][i])
  print(relevance_list[:5])
  p5,ap,rr= performance_metrics(relevance_list)
  # print(p5,ap,rr)
  m_p5.append(p5)
  m_ap.append(ap)
  m_rr.append(rr)

In [ ]:
print('Mean Precision @ 5: '+ str(np.mean(m_p5)))
print('Mean Average Precision: '+ str(np.mean(m_ap)))
print('Mean Reciprocal Rank: '+ str(np.mean(m_rr)))

Mean Precision @ 5: 0.17777777777777776
Mean Average Precision: 0.29748331527121763
Mean Reciprocal Rank: 0.40634987882808105


In [ ]:
QA_df.head()

,FaqID,FaqQuestion,FaqAnswer
0,8656,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...
1,40807,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...
2,57727,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...
3,120493,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne..."
4,62732,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.


In [ ]:
QA_df.loc[QA_df['FaqQuestion'] == "How to find about CORN burning stoves? Who sells such stoves & are they useful for New England residents? How costly for supply of fuel?", 'FaqID'].iloc[0]

92363

In [ ]:
QA_df.loc[QA_df['FaqQuestion'] == "How to find about CORN burning stoves? Who sells such stoves & are they useful for New England residents? How costly for supply of fuel?" , 'FaqID'].item()

92363